In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder .master('local[*]').appName('test').getOrCreate()
print(spark.version)
#spark.stop()

3.1.1

In [0]:
flights = spark.read.csv('/FileStore/tables/flights3.csv',header=True,inferSchema=True)
print("The data contain %d records." % flights.count())
flights.show(5)
print(flights.dtypes)

The data contain 50000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
 11| 20| 6| US| 19|JFK|2153| 9.48| 351| NA|
 0| 22| 2| UA| 1107|ORD| 316| 16.33| 82| 30|
 2| 20| 4| UA| 226|SFO| 337| 6.17| 82| -8|
 9| 13| 1| AA| 419|ORD|1236| 10.33| 195| -5|
 4| 2| 5| AA| 325|ORD| 258| 8.92| 65| NA|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows

[('mon', 'int'), ('dom', 'int'), ('dow', 'int'), ('carrier', 'string'), ('flight', 'int'), ('org', 'string'), ('mile', 'int'), ('depart', 'double'), ('duration', 'int'), ('delay', 'string')]

In [0]:
flights_drop_column = flights.drop('flight')
flights_drop_column.filter('delay IS NULL').count()

flights_valid_delay = flights_drop_column.filter('delay IS NULL')

flights_none_missing = flights_valid_delay.dropna()
print(flights_none_missing.count())

0

In [0]:
from pyspark.sql.functions import round

flights = flights.withColumn('km', round(flights.mile * 1.60934, 0)).drop('mile')

flights = flights.withColumn('label', (flights.delay >= 15).cast('integer'))

flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+------+-----+
mon|dom|dow|carrier|flight|org|depart|duration|delay| km|label|
+---+---+---+-------+------+---+------+--------+-----+------+-----+
 11| 20| 6| US| 19|JFK| 9.48| 351| NA|3465.0| null|
 0| 22| 2| UA| 1107|ORD| 16.33| 82| 30| 509.0| 1|
 2| 20| 4| UA| 226|SFO| 6.17| 82| -8| 542.0| 0|
 9| 13| 1| AA| 419|ORD| 10.33| 195| -5|1989.0| 0|
 4| 2| 5| AA| 325|ORD| 8.92| 65| NA| 415.0| null|
+---+---+---+-------+------+---+------+--------+-----+------+-----+
only showing top 5 rows

In [0]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = 'carrier', outputCol = 'carrier_idx')
indexer_model = indexer.fit(flights)
flights_indexed = indexer_model.transform(flights)

flights_indexed= StringIndexer(inputCol ='org', outputCol ='org_idx').fit(flights_indexed).transform(flights_indexed)

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=[ 'mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration'], outputCol='features')
flights_assembled = assembler.transform(flights_indexed)
flights_assembled.select('features', 'delay').show(5, truncate = False)

+-----------------------------------------+-----+
features |delay|
+-----------------------------------------+-----+
[11.0,20.0,6.0,6.0,2.0,3465.0,9.48,351.0]|NA |
[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0] |30 |
[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0] |-8 |
[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|-5 |
[4.0,2.0,5.0,1.0,0.0,415.0,8.92,65.0] |NA |
+-----------------------------------------+-----+
only showing top 5 rows

In [0]:
flights_assembled = flights_assembled.dropna()
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=17)

training_ratio = flights_train.count() / flights.count()
print(training_ratio)

0.7495

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

flights_train = flights_train.dropna()
Tree = DecisionTreeClassifier()
Tree_model = Tree.fit(flights_train)

prediction = Tree_model.transform(flights_test)
prediction.select('label','prediction','probability').show()

+-----+----------+--------------------+
label|prediction| probability|
+-----+----------+--------------------+
 0| 1.0|[0.35466470516121...|
 1| 1.0|[0.45092024539877...|
 1| 1.0|[0.35466470516121...|
 1| 0.0|[0.53230088495575...|
 1| 0.0|[0.53230088495575...|
 1| 1.0|[0.35466470516121...|
 1| 1.0|[0.35466470516121...|
 1| 1.0|[0.35466470516121...|
 1| 0.0|[0.75117924528301...|
 1| 0.0|[0.75117924528301...|
 1| 1.0|[0.35466470516121...|
 1| 1.0|[0.35466470516121...|
 1| 1.0|[0.35466470516121...|
 1| 1.0|[0.35466470516121...|
 0| 0.0|[0.53230088495575...|
 0| 1.0|[0.35466470516121...|
 0| 0.0|[0.75117924528301...|
 1| 1.0|[0.35466470516121...|
 0| 1.0|[0.35466470516121...|
 1| 1.0|[0.35466470516121...|
+-----+----------+--------------------+
only showing top 20 rows

In [0]:
prediction.groupBy('label','prediction').count().show()
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

accuracy = (TN+TP)/(TN+TP+FN+FP)
print(accuracy)

+-----+----------+-----+
label|prediction|count|
+-----+----------+-----+
 1| 0.0| 1355|
 0| 0.0| 2575|
 1| 1.0| 3526|
 0| 1.0| 2091|
+-----+----------+-----+

0.6390489158898083

In [0]:
from pyspark.ml.classification import LogisticRegression

logistic = LogisticRegression()
logistic_model = logistic.fit(flights_train)

prediction = logistic_model.transform(flights_test)
prediction.groupBy('label','prediction').count().show()

+-----+----------+-----+
label|prediction|count|
+-----+----------+-----+
 1| 0.0| 1680|
 0| 0.0| 2575|
 1| 1.0| 3201|
 0| 1.0| 2091|
+-----+----------+-----+

In [0]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(precision,recall)

from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction,{multi_evaluator.metricName:"weightedPrecision"})

binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction,{binary_evaluator.metricName:"areaUnderROC"})

print(weighted_precision,auc)

0.604875283446712 0.6558082360172096
0.6050195127140048 0.6424733342799964

In [0]:
from pyspark.ml.feature import OneHotEncoder

onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])
onehot = onehot.fit(flights_assembled)
flights_onehot = onehot.transform(flights_assembled)

flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
org|org_idx| org_dummy|
+---+-------+-------------+
ORD| 0.0|(7,[0],[1.0])|
SFO| 1.0|(7,[1],[1.0])|
JFK| 2.0|(7,[2],[1.0])|
LGA| 3.0|(7,[3],[1.0])|
SJC| 4.0|(7,[4],[1.0])|
SMF| 5.0|(7,[5],[1.0])|
TUS| 6.0|(7,[6],[1.0])|
OGG| 7.0| (7,[],[])|
+---+-------+-------------+

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

regression = LinearRegression(labelCol='duration').fit(flights_train)
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(10)
RegressionEvaluator(labelCol='duration').evaluate(predictions)

+--------+------------------+
duration| prediction|
+--------+------------------+
 370| 369.9999999999997|
 310| 310.0000000000004|
 130|130.00000000000017|
 250|249.99999999999952|
 230|230.00000000000028|
 85| 85.00000000000023|
 105|105.00000000000041|
 155|154.99999999999997|
 301| 301.0000000000005|
 315|315.00000000000057|
+--------+------------------+
only showing top 10 rows

Out[13]: 2.695777776881135e-13

In [0]:
inter = regression.intercept
print(inter)
coefs = regression.coefficients
print(coefs)

minutes_per_km = regression.coefficients[0]
print(minutes_per_km)

avg_speed = 60 / minutes_per_km
print(avg_speed)

1.0284708542301825e-12
[-6.557053215777649e-15,-3.701393309331552e-15,-1.5851250436425333e-14,2.5825153597022073e-14,-4.0055560646444513e-14,1.152557084512496e-15,-1.0429810541934474e-14,0.9999999999999846]
-6.557053215777649e-15
-9150451891350734.0

In [0]:
from pyspark.ml.feature import Bucketizer, OneHotEncoder

buckets = Bucketizer(splits=[0, 3, 6, 9, 12, 15, 18, 21, 24], inputCol='depart', outputCol='depart_bucket')

bucketed = buckets.transform(flights)
bucketed.select('depart', 'depart_bucket').show(5)

onehot = OneHotEncoder(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

flights_onehot = onehot.fit(bucketed).transform(bucketed)
flights_onehot.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+------+-------------+
depart|depart_bucket|
+------+-------------+
 9.48| 3.0|
 16.33| 5.0|
 6.17| 2.0|
 10.33| 3.0|
 8.92| 2.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
 9.48| 3.0|(7,[3],[1.0])|
 16.33| 5.0|(7,[5],[1.0])|
 6.17| 2.0|(7,[2],[1.0])|
 10.33| 3.0|(7,[3],[1.0])|
 8.92| 2.0|(7,[2],[1.0])|
+------+-------------+-------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
RegressionEvaluator(labelCol='duration').evaluate(predictions)

avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

avg_night_ogg = regression.intercept + regression.coefficients[6]
print(avg_night_ogg)

avg_night_jfk = regression.intercept + regression.coefficients[6] + regression.coefficients[3]
print(avg_night_jfk)

1.0284708542301825e-12
1.018041043688248e-12
1.04386619728527e-12

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)
prediction = regression.transform(flights_test)
rmse = RegressionEvaluator(labelCol='duration').evaluate(prediction)
print("The test RMSE is", rmse)

coeffs = regression.coefficients
print(coeffs)

The test RMSE is 0.9859516297254955
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9886166635388467]

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)

rmse = RegressionEvaluator(labelCol='duration').evaluate(regression.transform(flights_test))
print("The test RMSE is", rmse)

coeffs = regression.coefficients
print(coeffs)

zero_coeff = sum([beta == 0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 0.9859516297254955
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9886166635388467]
Number of coefficients equal to 0: 7

In [0]:
indexer = StringIndexer(inputCol='org', outputCol='org_idx')
onehot = OneHotEncoder(inputCols=['org_idx', 'dow'], outputCols=['org_dummy', 'dow_dummy'])

assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'], outputCol='features')

regression = LinearRegression(labelCol='duration')

In [0]:
from pyspark.ml import Pipeline

flights = spark.read.csv('/FileStore/tables/flights3.csv',header=True,inferSchema=True)
flights = flights.withColumn('km', round(flights.mile * 1.60934, 0)).drop('mile')
train,test = flights.randomSplit([0.8, 0.2], seed=17)

pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])
pipeline = pipeline.fit(train)
predictions = pipeline.transform(test)

In [0]:
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

params = ParamGridBuilder().build()

regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator,numFolds=5)
cv = cv.fit(flights_train)
pred = cv.transform(flights_test)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
indexer = StringIndexer(inputCol='org', outputCol='org_idx')
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

pipeline= Pipeline(stages =[indexer, onehot, assembler, regression])
cv = CrossValidator(estimator =pipeline, estimatorParamMaps =params,evaluator =evaluator)

In [0]:
params = ParamGridBuilder()
params = params.addGrid(regression.regParam, [0.01, 0.1, 1.0, 10.0]).addGrid(regression.elasticNetParam, [0.0, 0.5, 1.0])
params = params.build()
print('Number of models to be tested: ', len(params))

cv = CrossValidator(estimator =pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

Number of models to be tested: 12

In [0]:
model = cv.fit(train)
best_model = model.bestModel
print(best_model.stages)
best_model.stages[3].extractParamMap()

predictions = best_model.transform(test)
evaluator.evaluate(predictions)

[StringIndexerModel: uid=StringIndexer_933a04fe2b9e, handleInvalid=error, OneHotEncoderModel: uid=OneHotEncoder_e4983bcd3bdc, dropLast=true, handleInvalid=error, numInputCols=1, numOutputCols=1, VectorAssembler_1f003296d058, LinearRegressionModel: uid=LinearRegression_368003cc828b, numFeatures=8]
Out[24]: 11.165485694869778

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

tree = DecisionTreeClassifier().fit(flights_train)
gbt = GBTClassifier().fit(flights_train)
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(tree.transform(flights_test))
evaluator.evaluate(gbt.transform(flights_test))

print(gbt.getNumTrees)
print(gbt.featureImportances)

20
(8,[0,1,2,3,4,5,6,7],[0.20361259382530808,0.15329445248312915,0.1483230486718029,0.09792264126424094,0.16713405808242726,0.05754954585165519,0.1383423433209374,0.03382131650049909])

In [0]:
from pyspark.ml.classification import RandomForestClassifier
forest = RandomForestClassifier()

params = ParamGridBuilder() \
.addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
.addGrid(forest.maxDepth, [2, 5, 10]) \
.build()

evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

In [0]:
# Model
cv = cv.fit(flights_train)

# Average AUC for each parameter combination in grid
print(cv.avgMetrics)

# Average AUC for the best model
print(max(cv.avgMetrics))

# What's the optimal parameter value for maxDepth?
print(cv.bestModel.explainParam('maxDepth'))

# What's the optimal parameter value for featureSubsetStrategy?
print(cv.bestModel.explainParam('featureSubsetStrategy'))

# AUC for best model on testing data
print(evaluator.evaluate(cv.transform(flights_test)))

[0.6207043932733345, 0.6829876107241803, 0.7229003721638518, 0.6592291686434415, 0.68316434819817, 0.7190037283335078, 0.6592291686434415, 0.68316434819817, 0.7190037283335078, 0.6592291686434415, 0.68316434819817, 0.7190037283335078]
0.7229003721638518
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5, current: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: auto, current: all)
0.730828919014069